<a href="https://colab.research.google.com/github/thogood212/dev_matching2022/blob/main/2022_Dev_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#시계열 데이터를 사용한 행동 분류

##[목표] 주어진 데이터를 활용하여 F1 score(macro)를 최대로 높이세요.



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report

import pandas as pd
import numpy as np
import os

In [2]:
batch_size = 256
lr = 0.0001

epochs = 500
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


In [3]:
dataset0_classes = os.listdir('/content/drive/MyDrive/dev_ML/dataset0/train/')
dataset0_label_encoder = LabelEncoder()

dataset0_label_encoder.fit(dataset0_classes)
print(dataset0_label_encoder.classes_)
print(len(dataset0_label_encoder.classes_))

dataset1_classes = os.listdir('/content/drive/MyDrive/dev_ML/dataset1/train/')
dataset1_label_encoder = LabelEncoder()

dataset1_label_encoder.fit(dataset1_classes)
print(dataset1_label_encoder.classes_)
print(len(dataset1_label_encoder.classes_))

['class0' 'class1' 'class11' 'class12' 'class13' 'class14' 'class15'
 'class2' 'class3' 'class4' 'class5' 'class6' 'class7' 'class8' 'class9']
15
['class0' 'class1' 'class10' 'class2' 'class3' 'class4' 'class5' 'class6'
 'class7' 'class8' 'class9']
11


##데이터셋/데이터로더

In [4]:
def create_dataset(dataset_dir):
  X, y = [], []
  labels = os.listdir(dataset_dir)
  for label in labels:
    file_list = os.listdir(dataset_dir + label + '/')
    for f in file_list:
      temp = pd.read_csv(dataset_dir + label + '/' + f)
      X.append(torch.from_numpy(temp.values))
      y.append(label)
  X = pad_sequence(X, batch_first=True)
  return X, y

In [5]:
#데이터셋 주소
dataset0_train_dir = '/content/drive/MyDrive/dev_ML/dataset0/train/'
dataset0_test_dir = '/content/drive/MyDrive/dev_ML/dataset0/test/'
dataset1_train_dir = '/content/drive/MyDrive/dev_ML/dataset1/train/'
dataset1_test_dir = '/content/drive/MyDrive/dev_ML/dataset1/test/'

In [6]:
# dataset0's training/test datasets & dataloader
dataset0_X_train, dataset0_y_train = create_dataset(dataset0_train_dir)
dataset0_y_train = dataset0_label_encoder.transform(dataset0_y_train)

dataset0_X_test, dataset0_y_test = create_dataset(dataset0_test_dir)
dataset0_y_test = dataset0_label_encoder.transform(dataset0_y_test)

dataset0_train_dataset = TensorDataset(torch.tensor(dataset0_X_train).float(), torch.from_numpy(dataset0_y_train))
dataset0_test_dataset = TensorDataset(torch.tensor(dataset0_X_test).float(), torch.from_numpy(dataset0_y_test))

dataset0_train_dataloader = DataLoader(dataset0_train_dataset,
                                       batch_size=batch_size)
dataset0_test_dataloader= DataLoader(dataset0_test_dataset,
                                     batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


In [7]:
# dataset1's training datasets & dataloader
dataset1_X_train, dataset1_y_train = create_dataset(dataset1_train_dir)
dataset1_y_train = dataset1_label_encoder.transform(dataset1_y_train)

dataset1_train_dataset = TensorDataset(torch.tensor(dataset1_X_train).float(), torch.from_numpy(dataset1_y_train))
dataset1_train_dataloader = DataLoader(dataset1_train_dataset,
                                       batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [8]:
"""
해당 셀의 코드 중 dataloader 부분의 `shuffle=False` 는 수정하면 안됩니다.
"""

dataset1_test_X = []
test_list = os.listdir(dataset1_test_dir)
for f in test_list:
  temp = pd.read_csv(dataset1_test_dir + f)
  dataset1_test_X.append(torch.from_numpy(temp.values))

dataset1_test_X = pad_sequence(dataset1_test_X, batch_first=True)
dataset1_test_dataset = TensorDataset(dataset1_test_X)
dataset1_test_dataloader = DataLoader(dataset1_test_dataset, batch_size=batch_size, shuffle=False)

##EDA


In [ ]:
#사이즈 확인
#3차원
for x,y in dataset0_train_dataloader:
  print(x.size(),y.size())
  break

torch.Size([256, 696, 6]) torch.Size([256])


In [ ]:
#사이즈 확인
for x,y in dataset1_train_dataloader:
  print(x.size(),y.size())
  break

torch.Size([256, 800, 6]) torch.Size([256])


In [32]:
#dataset0 _ 훈련데이터 확인 (총12414개,696개의 텐서, 텐서는 6개의 값으로 구성)== 데이터 1개당 각각의 클래스 정보(라벨)를 가지고 696개의 메타데이터(요소)가 6개의 시간별로 데이터를 이룸
#
dataset0_train_dataset

(tensor([[ 1.0405e+00,  9.8331e+00,  9.6893e-02,  7.0250e-03,  3.0849e-02,
           1.8937e-02],
         [ 1.0386e+00,  9.8345e+00,  1.0017e-01,  1.1912e-02,  3.6041e-02,
           1.3439e-02],
         [ 1.0364e+00,  9.8362e+00,  1.0408e-01,  1.2217e-02,  4.0012e-02,
           1.4661e-02],
         ...,
         [ 7.2082e+00,  1.3439e+00,  6.3675e+00, -3.0543e-04, -1.4355e-02,
           6.4141e-03],
         [ 7.2011e+00,  1.3510e+00,  6.3652e+00,  1.5272e-03, -1.1912e-02,
           3.6652e-03],
         [ 7.1944e+00,  1.3577e+00,  6.3629e+00, -3.6652e-03, -5.1924e-03,
           1.5272e-03]]), tensor(10))

In [ ]:
dataset0_X_train[0]

tensor([-4.4663e+00,  2.3236e+00,  8.1045e+00, -4.2455e-02,  9.1630e-04,
         3.0849e-02], dtype=torch.float64)

##CNN+LSTM

In [76]:
m = nn.Conv1d(in_channels=696 ,out_channels=256,kernel_size=2)
n = nn.Conv1d(in_channels=256 ,out_channels=128,kernel_size=2)
x = nn.Conv1d(in_channels=128 ,out_channels=64,kernel_size=2)
y = nn.Conv1d(in_channels=64 ,out_channels=32,kernel_size=2)
z = nn.Flatten()
lstm = nn.LSTM(input_size=64,
                            hidden_size=32,
                            num_layers=2,
                            bias=True,
                            bidirectional=True,
                            batch_first=True)
fc_1 = nn.Linear(64, 32)
fc_2 = nn.Linear(32, len(dataset0_label_encoder.classes_))

input = torch.randn(256, 696, 6)
output1 = m(input)
output2 = n(output1)
output3 = x(output2)
output4 = y(output3)
output5 = z(output4)
output6, hid = lstm(output5)
output6 = output6.view(64, -1)
output7 = fc_1(output6)
output8 = fc_2(output7)
#output = output.view(32,-1)

RuntimeError: ignored

In [52]:
output1.size()

torch.Size([256, 256, 5])

In [51]:
output2.size()

torch.Size([256, 128, 4])

In [50]:
output3.size()

torch.Size([256, 64, 3])

In [55]:
output4.size()

torch.Size([256, 32, 2])

In [57]:
output5.size()

torch.Size([256, 64])

In [61]:
output6.size(),len(hid)

(torch.Size([256, 32]), 2)

In [65]:
output6.size(),len(hid)

(torch.Size([256, 64]), 2)

In [74]:
output7.size()

torch.Size([256, 32])

In [84]:
output8.size()

torch.Size([256, 15])

In [83]:
#lstm 예시
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)

RuntimeError: ignored

In [82]:
output.size()

torch.Size([5, 3, 20])

In [112]:
# 모델 생성 작성
batch_size = 256
Sequence = 800
kernel_size = 2

class model_cnn(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(model_cnn,self).__init__()
        #800,6 -> 256,6
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.conv1d1 = nn.Conv1d(in_channels=seq_length,out_channels=256,kernel_size=2)
        #self.maxpooling1 = nn.MaxPool1d(kernel_size)
        self.conv1d2 = nn.Conv1d(in_channels=256 ,out_channels=128,kernel_size=2)
        self.conv1d3 = nn.Conv1d(in_channels=128 ,out_channels=64,kernel_size=2)
        self.conv1d4 = nn.Conv1d(in_channels=64 ,out_channels=32,kernel_size=2)
        #self.lstm = nn.LSTM(1,800,256)
        #self.maxpooling2 = nn.MaxPool1d(kernel_size)
        self.relu = nn.ReLU(inplace=True)
        self.flatten = nn.Flatten()
        self.lstm = nn.LSTM(input_size=64,
                            hidden_size=32,
                            num_layers=2,
                            bias=True,
                            bidirectional=True,
                            batch_first=True)
        self.fc1 = nn.Linear(64,32)
        self.fc2 = nn.Linear(32,num_classes)
        self.softmax = nn.Softmax()

    def forward(self,x):
        x = self.conv1d1(x)
        x = self.relu(x)
        #x = self.maxpooling1(x)
        x = self.conv1d2(x)
        x = self.relu(x)
        x = self.conv1d3(x)
        x = self.relu(x)
        x = self.conv1d4(x)
        x = self.relu(x)
        x = self.flatten(x)
        x,hid = self.lstm(x)
        #x = x.view(-1,64)
        x = self.relu(x)
        #x = self.relu(x[0])
        #x = x.view(64, -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        #x = self.fc3(x)
        #x = self.relu(x)
        x = self.softmax(x)
        
        return x


In [113]:
num_classes = 15
input_size = 64
hidden_size = 32
num_layers = 2
seq_length = 696

model_cnn=model_cnn(num_classes, input_size, hidden_size, num_layers, seq_length)
print(model_cnn)

model_cnn(
  (conv1d1): Conv1d(696, 256, kernel_size=(2,), stride=(1,))
  (conv1d2): Conv1d(256, 128, kernel_size=(2,), stride=(1,))
  (conv1d3): Conv1d(128, 64, kernel_size=(2,), stride=(1,))
  (conv1d4): Conv1d(64, 32, kernel_size=(2,), stride=(1,))
  (relu): ReLU(inplace=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (lstm): LSTM(64, 32, num_layers=2, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=15, bias=True)
  (softmax): Softmax(dim=None)
)


In [114]:
model_cnn = model_cnn.to(device)
optimizer = torch.optim.Adam(model_cnn.parameters(), lr=1e-5)

In [188]:
train_losses = []
valid_losses = []

def Train(model, data_loader):
    
    running_loss = .0
    
    model.train()
    model = model.to(device)


    for idx, (inputs, labels) in enumerate(data_loader):
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        print(1)
        preds = model(inputs.float())
        preds = preds.argmax(dim=1).to(device)
        #preds = preds.cpu().numpy()
        #labels = labels.cpu().numpy()
        #print(preds)
        #print(labels)
        loss = f1_score(labels, preds, average='macro')
        print(1)
        loss = torch.tensor(loss).to(device)
        loss.backward()
        print(1)
        optimizer.step()
        print(1)
        running_loss += loss
        
    train_loss = running_loss/len(data_loader)
    train_losses.append(train_loss.detach().numpy())
    
    print(f'train_loss {train_loss}')

In [189]:
for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    Train(model_cnn, dataset0_train_dataloader)

epochs 1/500
1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


TypeError: ignored

In [ ]:
# dataset1의 test 데이터를 사용한 일반화 성능 확인

model_cnn.eval()
predicted = []
with torch.no_grad():
  for idx, x in enumerate(dataset1_test_dataloader):
    x = x.permute(0, 2, 1).contiguous().to(device)

    optimizer.zero_grad()
    output = model_cnn(x.float())

    _, preds = torch.max(output, 1)
    predicted.extend(preds.cpu().numpy())
  torch.cuda.empty_cache()

In [ ]:
pd_preds = pd.DataFrame(predicted, columns=['predicted value'])
pd_preds.to_csv('submission.csv')
pd_preds.head()

In [ ]:
## 변경해야함
    def __init__(self, hidden_size: int, n_label: int):
        super(CustomClassifier, self).__init__()

        self.bert = BertModel.from_pretrained("klue/bert-base")

        dropout_rate = 0.1
        linear_layer_hidden_size = 32

        self.classifier = nn.Sequential(
        nn.Linear(hidden_size, linear_layer_hidden_size),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(linear_layer_hidden_size, n_label)
        )

        elif args.model == "transformer":

        hidden_dims = args.hidden_dims # 256
        n_heads = args.n_heads # 8
        n_layers = args.n_layers # 6
        len_max_seq = args.samplet
        dropout = args.dropout
        d_inner = hidden_dims*4

        model = TransformerEncoder(in_channels=args.input_dims, len_max_seq=len_max_seq,
            d_word_vec=hidden_dims, d_model=hidden_dims, d_inner=d_inner,
            n_layers=n_layers, n_head=n_heads, d_k=hidden_dims//n_heads, d_v=hidden_dims//n_heads,
            dropout=dropout, nclasses=args.nclasses)

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        
        # BERT 모델의 마지막 레이어의 첫번재 토큰을 인덱싱
        last_hidden_states = outputs[0] # last hidden states (batch_size, sequence_len, hidden_size)
        cls_token_last_hidden_states = last_hidden_states[:,0,:] # (batch_size, first_token, hidden_size)

        logits = self.classifier(cls_token_last_hidden_states)

        return logits

##(2)Transformer


In [ ]:
# 모델 생성 작성
class model(nn.Module):
  def __init__(self, **kargs):
    super(model, self).__init__()

    self.transformer = TransformerEncoder(in_channels=args.input_dims, len_max_seq=len_max_seq,
            d_word_vec=hidden_dims, d_model=hidden_dims, d_inner=d_inner,
            n_layers=n_layers, n_head=n_heads, d_k=hidden_dims//n_heads, d_v=hidden_dims//n_heads,
            dropout=dropout, nclasses=args.nclasses)
    
  def forward(self, x):
    """
    코드 작성하세요
    """
    return output



model = model(**kargs).to(device)
print(model)